<a href="https://colab.research.google.com/github/mitul01/FER/blob/main/FER_manual_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/colab-20210303T131708Z-001/colab

/content/drive/MyDrive/colab-20210303T131708Z-001/colab


In [2]:
from config import emotion_config as config
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
from pipeline.io import HDF5DatasetGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import load_model
from keras import backend as K
import argparse
import os

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout,Activation
from tensorflow.keras import Sequential

In [3]:
trainAug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.1,
    horizontal_flip = True, rescale = 1 / 255.0, fill_mode = "nearest")

valAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [4]:
model = Sequential()
model.add(Conv2D(64, (1, 1), padding='same', activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add((Conv2D(128, (3, 3),padding='same', activation='relu')))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(128, (1, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 1),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3,3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))
model.summary()


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model.fit(X_train, Y_train, batch_size=64, epochs=40, steps_per_epoch=(len(X_train)/128))


#model.fit(X_train, Y_train, batch_size=64, epochs=40, steps_per_epoch=len(X_train)/128, validation_split = 0.25)
batch_size = 32

epochs = 100
from tensorflow.keras import optimizers
optim=optimizers.Adam(0.001)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=7,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // batch_size,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    verbose = 1
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 128)       4

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


897/897 [==============================] - 192s 175ms/step - loss: 1.8785 - accuracy: 0.2575 - val_loss: 1.4020 - val_accuracy: 0.4423
Epoch 2/100
897/897 [==============================] - 159s 177ms/step - loss: 1.4155 - accuracy: 0.4387 - val_loss: 1.2106 - val_accuracy: 0.5216
Epoch 3/100
897/897 [==============================] - 158s 176ms/step - loss: 1.2507 - accuracy: 0.5123 - val_loss: 1.1273 - val_accuracy: 0.5692
Epoch 4/100
897/897 [==============================] - 158s 176ms/step - loss: 1.1822 - accuracy: 0.5415 - val_loss: 1.0778 - val_accuracy: 0.5830
Epoch 5/100
897/897 [==============================] - 158s 176ms/step - loss: 1.1381 - accuracy: 0.5630 - val_loss: 1.0377 - val_accuracy: 0.6026
Epoch 6/100
897/897 [==============================] - 158s 176ms/step - loss: 1.0989 - accuracy: 0.5773 - val_loss: 0.9887 - val_accuracy: 0.6223
Epoch 7/100
897/897 [==============================] - 158s 176ms/step - loss: 1.0632 - accuracy: 0.5931 - val_loss: 0.9697 - val_

In [8]:
optim=optimizers.Adam(0.00001)

model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // batch_size,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // batch_size,
    epochs = 10,
    callbacks = callbacks,
    verbose = 1
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
897/897 [==============================] - 160s 177ms/step - loss: 0.7957 - accuracy: 0.7018 - val_loss: 0.8891 - val_accuracy: 0.6748
Epoch 2/10
897/897 [==============================] - 158s 176ms/step - loss: 0.8015 - accuracy: 0.6997 - val_loss: 0.8961 - val_accuracy: 0.6732
Epoch 3/10
897/897 [==============================] - 158s 176ms/step - loss: 0.8028 - accuracy: 0.6952 - val_loss: 0.8946 - val_accuracy: 0.6731
Epoch 4/10
897/897 [==============================] - 158s 176ms/step - loss: 0.7963 - accuracy: 0.7011 - val_loss: 0.8944 - val_accuracy: 0.6728

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 5/10
897/897 [==============================] - 158s 176ms/step - loss: 0.7941 - accuracy: 0.7020 - val_loss: 0.8978 - val_accuracy: 0.6731
Epoch 6/10
897/897 [==============================] - 158s 176ms/step - loss: 0.7992 - accuracy: 0.6996 - val_loss: 0.8933 - val_accuracy: 0.6744
Epoch 7/10
897/897 [=======================

In [9]:
testAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

testGen = HDF5DatasetGenerator(config.TEST_HDF5, config.BATCH_SIZE,
    aug = testAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [10]:
predictions = model.predict_generator(testGen.generator(),steps=testGen.numImages //64, max_queue_size=64*2)

(loss, acc) = model.evaluate_generator(
	testGen.generator(),
	steps=testGen.numImages // config.BATCH_SIZE,
	max_queue_size=config.BATCH_SIZE * 2)
print("[INFO] accuracy: {:.2f}".format(acc * 100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[INFO] accuracy: 65.04
